# Exploring the public IBL data with ONE
This tutorial will give an introduction into the publically available data and show how to download electrophysiology data for a probe insertion and display a simple raster plot

First let's get started by importing the ONE module and connecting to the database

In [1]:
# Import one and connect to the database
from one.api import ONE
one = ONE(base_url='https://openalyx.internationalbrainlab.org', silent=True)

We want to see what probe insertions are available on the database. To list them we can use the following command

In [2]:
probe_insertions = one.alyx.rest('insertions', 'list')

We can find the number of probes insertions available and some information about the first probe

In [3]:
from pprint import pprint
print(f'N probes = {len(probe_insertions)}')
pprint(probe_insertions[0])

N probes = 4
{'auto_datetime': '2021-04-02T15:03:46.367818',
 'datasets': ['8091108f-a0d2-44e0-bf01-5c0a93de7844',
              '54e5d55d-e19d-452d-beed-ff49a5b3b38b',
              'eaed8cf5-0bb9-4eaf-8d8e-3471e39d63fa',
              '236a116c-8a18-4dd0-a2b5-169b28ef7fd4',
              'a3bf84f1-3ee2-4f3c-a36a-4b4959a513ba',
              'bce68e1c-1d72-4bf9-b77f-0c9e3b1c5c12',
              'c0c3c1a7-ee4d-4117-8a6f-93d9d43c52ee',
              'e9d8ddb5-ddaf-48b7-8a78-49d909c80ef1',
              '138676a2-d310-49b4-865e-0e14f8b13182',
              '837af07a-26aa-4264-97d6-fa55d2382f77',
              '4f397701-da2e-460b-9937-a6a53ff23959',
              '45854a1c-c309-4bb0-806e-a4f9f1617472',
              'b31f4721-0163-4aec-a92e-7353764ce355',
              '25c2b0f4-6201-407d-8cf6-f892370a5a44',
              'ffea1a62-dc7e-4d01-8f86-03801a88615f',
              'eaa07c05-df66-42ce-ae59-326b6ac9e8f0',
              '4420d325-141b-4d30-8b92-0f8f545274d3',
              '6a0c68

Each probe insertion has a unique ID (pid) that we can use to search the database and find the datasets that are linked to it. Let's look at the datasets associated with the first probe in the list

In [4]:
pid = probe_insertions[0]['id']

datasets = one.alyx.rest('datasets', 'list', probe_insertion=pid)
print(f'N datasets = {len(datasets)}')
pprint(datasets[0])

N datasets = 44
{'auto_datetime': '2021-02-10T20:24:31.484939',
 'collection': 'alf/probe00',
 'created_by': 'afb0fdad',
 'created_datetime': '2020-09-22T09:31:52.393857',
 'data_format': 'csv',
 'dataset_type': 'clusters.uuids',
 'default_dataset': True,
 'experiment_number': 1,
 'file_records': [{'data_repository': 'flatiron_hoferlab',
                   'data_repository_path': '/public/hoferlab/Subjects/',
                   'data_url': 'https://ibl.flatironinstitute.org/public/hoferlab/Subjects/SWC_043/2020-09-21/001/alf/probe00/clusters.uuids.3d9cbd53-fe3f-41ca-ae37-71936f5ca68f.csv',
                   'exists': True,
                   'id': 'ae3974e1-7cdc-42cb-8632-b9ae0391e71d',
                   'relative_path': 'SWC_043/2020-09-21/001/alf/probe00/clusters.uuids.csv'}],
 'file_size': 29383,
 'hash': '41ec5a83b7d7f41df9730fcae9cc7024',
 'name': 'clusters.uuids.csv',
 'protected': False,
 'public': False,
 'revision': None,
 'session': 'https://openalyx.internationalbrainlab.o

To get a more concise summary of the datasets available we can loop through and display their names

In [5]:
dataset_names = [dset['name'] for dset in datasets]
pprint(dataset_names)

['clusters.uuids.csv',
 'spikes.clusters.npy',
 'spikes.times.npy',
 'spikes.samples.npy',
 '_kilosort_raw.output.tar',
 'clusters.amps.npy',
 '_spikeglx_sync.channels.probe00.npy',
 '_spikeglx_sync.times.probe00.npy',
 'spikes.amps.npy',
 'clusters.metrics.pqt',
 'clusters.waveformsChannels.npy',
 'clusters.waveforms.npy',
 'clusters.peakToTrough.npy',
 'channels.rawInd.npy',
 '_spikeglx_sync.polarities.probe00.npy',
 'clusters.depths.npy',
 'spikes.depths.npy',
 'clusters.channels.npy',
 '_spikeglx_ephysData_g0_t0.imec0.ap.cbin',
 '_spikeglx_ephysData_g0_t0.imec0.lf.cbin',
 '_spikeglx_ephysData_g0_t0.imec0.lf.ch',
 '_spikeglx_ephysData_g0_t0.imec0.ap.ch',
 '_spikeglx_ephysData_g0_t0.imec0.ap.meta',
 '_spikeglx_ephysData_g0_t0.imec0.lf.meta',
 '_spikeglx_ephysData_g0_t0.imec0.sync.npy',
 '_spikeglx_ephysData_g0_t0.imec0.timestamps.npy',
 '_spikeglx_ephysData_g0_t0.imec0.wiring.json',
 'spikes.templates.npy',
 'channels.localCoordinates.npy',
 'templates.waveformsChannels.npy',
 'templ

We are interested in downloading the spikes datasets output from spike sorting in order to display a raster plot of the neural activity recorded on the probe. To download the data we can use the following approach

In [6]:
dtypes = ['spikes.times',
          'spikes.depths',
          'spikes.amps']
eid, probe = one.pid2eid(pid)
spikes = one.load_object(eid, 'spikes', collection=f'alf/{probe_insertions[0]["name"]}',attribute=['times', 'depths',
                                                                                                   'amps'])

We can import a module from brainbox that plots the raster across time

In [7]:
from brainbox.plot import driftmap
driftmap(spikes['times'], spikes['depths'], t_bin=0.1, d_bin=5)

ModuleNotFoundError: No module named 'brainbox'